In [1]:
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import sqlite3 as sql

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.5.0'

In [4]:
# employee = pd.read_csv('employee.csv')
# employee.head()

con = sql.connect("data/hr.sqlite")
# con.row_factory = sql.Row
# cur = con.cursor()
employee = pd.read_sql_query("select is_married, marital_status_id, employee_status_id, department_id, perf_score_id, is_diversity_job_fair, employee_salary, is_active, is_terminated, position_id, is_underrep_gender, is_citizen, gender_id, is_hispanic_latino, is_underrep_race_eth, survey_engagement_score, survey_emp_satisfaction_score, count_special_projects, count_days_late_past_30_days, count_absences, employee_status from employee", con)

# dataset = cur.fetchall()
con.close()


In [5]:
employee

,is_married,marital_status_id,employee_status_id,department_id,perf_score_id,is_diversity_job_fair,employee_salary,is_active,is_terminated,position_id,...,is_citizen,gender_id,is_hispanic_latino,is_underrep_race_eth,survey_engagement_score,survey_emp_satisfaction_score,count_special_projects,count_days_late_past_30_days,count_absences,employee_status
0,0,0,1,5,3.0,0,65288,1,0,21,...,1,1,0,0,3.19,3.0,0,0,9,Active
1,0,0,1,5,3.0,0,66738,1,0,21,...,1,0,0,0,4.53,5.0,0,0,5,Active
2,1,1,3,5,3.0,0,50428,0,1,20,...,1,1,0,1,5.00,3.0,0,0,11,Voluntarily Terminated
3,1,1,3,3,3.0,0,75281,0,1,17,...,1,1,0,0,5.00,3.0,5,0,11,Voluntarily Terminated
4,1,1,1,2,3.0,1,63000,1,0,1,...,1,0,0,1,4.50,2.0,6,0,14,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,0,0,1,6,3.0,0,58370,1,0,3,...,1,0,0,1,3.69,3.0,0,0,18,Active
308,0,0,1,6,1.0,0,59231,1,0,3,...,1,0,1,0,2.30,1.0,0,2,17,Active
309,1,1,3,5,3.0,1,54670,0,1,20,...,1,0,1,1,4.20,4.0,0,0,12,Voluntarily Terminated
310,0,2,3,5,3.0,0,52505,0,1,20,...,1,1,0,0,5.00,5.0,0,0,1,Voluntarily Terminated


In [6]:

X = employee.drop("employee_status", axis=1)
y = employee["employee_status"]
print(X.shape, y.shape)

(312, 20) (312,)


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [9]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [11]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20 ))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 12,503
Trainable params: 12,503
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=60,
    shuffle=True,
    verbose=2
)

Epoch 1/60
8/8 - 1s - loss: 0.8953 - accuracy: 0.6410
Epoch 2/60
8/8 - 0s - loss: 0.5621 - accuracy: 0.7393
Epoch 3/60
8/8 - 0s - loss: 0.3781 - accuracy: 0.9444
Epoch 4/60
8/8 - 0s - loss: 0.2671 - accuracy: 0.9530
Epoch 5/60
8/8 - 0s - loss: 0.2010 - accuracy: 0.9530
Epoch 6/60
8/8 - 0s - loss: 0.1632 - accuracy: 0.9530
Epoch 7/60
8/8 - 0s - loss: 0.1357 - accuracy: 0.9530
Epoch 8/60
8/8 - 0s - loss: 0.1186 - accuracy: 0.9530
Epoch 9/60
8/8 - 0s - loss: 0.1073 - accuracy: 0.9530
Epoch 10/60
8/8 - 0s - loss: 0.0964 - accuracy: 0.9530
Epoch 11/60
8/8 - 0s - loss: 0.0887 - accuracy: 0.9530
Epoch 12/60
8/8 - 0s - loss: 0.0801 - accuracy: 0.9615
Epoch 13/60
8/8 - 0s - loss: 0.0732 - accuracy: 0.9658
Epoch 14/60
8/8 - 0s - loss: 0.0657 - accuracy: 0.9701
Epoch 15/60
8/8 - 0s - loss: 0.0610 - accuracy: 0.9829
Epoch 16/60
8/8 - 0s - loss: 0.0532 - accuracy: 0.9829
Epoch 17/60
8/8 - 0s - loss: 0.0562 - accuracy: 0.9744
Epoch 18/60
8/8 - 0s - loss: 0.0426 - accuracy: 0.9915
Epoch 19/60
8/8 - 0

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: nan - accuracy: 0.9872
Normal Neural Network - Loss: nan, Accuracy: 0.9871794581413269


In [18]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

/Users/stevenzych/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [19]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: ['Active' 'Voluntarily Terminated' 'Voluntarily Terminated' 'Active'
 'Active']
Actual Labels: ['Active', 'Voluntarily Terminated', 'Voluntarily Terminated', 'Active', 'Active']


In [20]:
# con = sql.connect("data/hr.sqlite")
# con.row_factory = sql.Row
# cur = con.cursor()
# cur.execute("select is_married, gender_id, is_hispanic_latino,survey_engagement_score, count_absences, employee_status from employee")
# dataset = cur.fetchall()